# Mosaic AI Agent Evaluation

### Loading and testing the model serving

In [0]:
import os
import requests
import pandas as pd
import json

os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get(scope="poc_scope", key="accessForDatabricksAPI")

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = 'https://adb-2411896520795414.14.azuredatabricks.net/serving-endpoints/Chatbot_poc/invocations'
    headers = {'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN")}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

# Correct column name to match model schema
test_input = pd.DataFrame({"query": ["What should be done if the washer is installed in a location with freezing temperatures?"]})  
score_model(test_input)

{'predictions': '1. Turn off the washer and unplug the power cord.\n2. Pour warm water over the water tap to loosen the water hose.\n3. Disconnect the water hose and soak it in warm water.\n4. Pour warm water into the drum and leave it for about 10 minutes.\n5. Reconnect the water hose to the water tap.\n\nIf the washer still doesn’t operate normally, repeat the steps above.'}

In [0]:
%pip install mlflow databricks-agents

%pip install -r /local_disk0/repl_tmp_data/ReplId-1958f-66895-3/tmpsvlmv04c/requirements.txt
dbutils.library.restartPython()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.5/2.5 MB 203.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.0/1.0 MB 213.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/415.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.7/415.7 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.30
    Not uninstalling langchain-core at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-63dc9d33-c60f-4d04-b330-11c04c8

### loading pyfunc model

In [0]:
import mlflow.pyfunc
import os
import pandas as pd
mlflow.set_registry_uri("databricks-uc")
os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get(scope="poc_scope", key="accessForDatabricksAPI")
# Load the model from Unity Catalog
model_name = "genai-poc-catalog.rag.appliance_chatbot_pyfunc"
# mlflow.pyfunc.get_model_dependencies(f"models:/{model_name}/9")
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/9")
dataset = pd.DataFrame({"query": ["What is the purpose of the horizon temperature monitor in the refrigerator?"]})  
loaded_model.predict(dataset)

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/root/.ipykernel/2190/command-5173890031752958-3062494590:50: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


'I\'m sorry for any confusion, but the information provided doesn\'t mention a "horizon temperature monitor" in a refrigerator. It\'s possible that there might be a misunderstanding or a typo in the question. The text you provided discusses a washing machine and its various features, troubleshooting, and maintenance. If you have any questions related to the washing machine or any other home appliance, I\'d be happy to help!'

Trace(request_id=tr-d0b75f0891ff4155a983239b8c90c546)

### Evaluation sample

In [0]:
import mlflow
import pandas as pd

# Define test examples
eval_set = pd.DataFrame({
    "request": [{"query": "What should be done if the washer is installed in a location with freezing temperatures?"},
                {"query": "What is the purpose of the horizon temperature monitor in the refrigerator?"},
                {"query": "What does a SUDS message mean?"},
                {"query": "Why is it important to run the calibration cycle before using the washer?"}]
    })

# Define global guidelines
global_guidelines = {
  "english": ["The response must be in English"],
  "clarity": ["The response must be clear, coherent, and concise"],
}

# Ensure `loaded_model` is defined before calling mlflow.evaluate
evaluation_results = mlflow.evaluate(
    model=loaded_model,  # Ensure this variable is defined
    data=eval_set,
    model_type="databricks-agent",  # Replace with your model type
    evaluator_config={
       "databricks-agent": {"global_guidelines": global_guidelines}
    }
)

display(evaluation_results)


/local_disk0/.ephemeral_nfs/envs/pythonEnv-63dc9d33-c60f-4d04-b330-11c04c89ff54/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/03/13 12:14:37 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/4 [Elapsed: 00:00, Remaining: ?]

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 1/4 pass

In [0]:
eval_set = [
    {
        "query": "What safety precautions should be taken when using the washer?"
    },
    {
        "query": "Why is it important to remove the washer door before discarding the appliance?"
    },
    {
        "query": "What does the CALIFORNIA PROPOSITION 65 WARNING indicate in the washer manual?"
    },
    {
        "query": "What should be done if the washer is installed in a location with freezing temperatures?"
    },
    {
        "query": "Why is it important to run the calibration cycle before using the washer?"
    },
    {
        "query": "How does reduceByKey differ from groupByKey in Spark?"
    },
    {
        "query": "What should be done if the washer emits a burning smell or smoke?"
    },
    {
        "query": "How should detergent be used in this washer?"
    },
    {
        "query": "What are the critical installation warnings mentioned in the manual?"
    },
    {
        "query": "What are the common troubleshooting steps if the washer is not draining?"
    },
    {
        "query": "What special features does the washer include?"
    },
    {
        "query": "What should you do before the first use of the washer?"
    },
    {
        "query": "What does the Self Clean+ function do?"
    },
    {
        "query": "What is the purpose of the emergency drain feature?"
    },
    {
        "query": "What should you do if your washer is exposed to a gas leak?"
    },
    {
        "query": "What are the regulatory notices included in the manual?"
    },
    {
        "query": "How does the washer ensure environmental protection?"
    },
    {
        "query": "How does SmartThings enhance the washer\u2019s functionality?"
    },
    {
        "query": "What should be done during an extended period of non-use?"
    },
    {
        "query": "What information is provided in the fabric care chart?"
    },
    {
        "query": "Why should children not be allowed to play inside the washer?"
    },
    {
        "query": "What should be removed before using the washer for the first time?"
    },
    {
        "query": "Why is it dangerous to wash clothes contaminated with gasoline or alcohol?"
    },
    {
        "query": "What can happen if the washer door is forcibly opened during operation?"
    },
    {
        "query": "Why should you not insert your hand under the washer while it is operating?"
    },
    {
        "query": "What is the risk of touching the power plug with wet hands?"
    },
    {
        "query": "What should you do if the water supply hose comes loose and floods the appliance?"
    },
    {
        "query": "What is the proper way to unplug the appliance?"
    },
    {
        "query": "Why should children not climb onto the washer?"
    },
    {
        "query": "Why is it dangerous to use an inappropriate fuse in the washer?"
    },
    {
        "query": "Why should the power plug be unplugged during a thunderstorm?"
    },
    {
        "query": "What should you do if a foreign substance enters the washer?"
    },
    {
        "query": "Why should you clean the washer if it gets contaminated with detergent or food waste?"
    },
    {
        "query": "What precaution should be taken after a long period of non-use before opening the water supply?"
    },
    {
        "query": "What might happen if a draining error occurs and the washer continues running?"
    },
    {
        "query": "Why is it important to fully insert laundry into the washer?"
    },
    {
        "query": "Why should you turn off the water supply when the washer is not in use?"
    },
    {
        "query": "What should you check before using the washer to avoid water leaks?"
    },
    {
        "query": "Why should you avoid placing objects like candles or chemicals on the washer?"
    },
    {
        "query": "What is the risk of spraying insecticide on the washer?"
    },
    {
        "query": "Why should you avoid touching the drained water from a high-temperature wash?"
    },
    {
        "query": "Why should waterproof items not be washed unless there is a special cycle?"
    },
    {
        "query": "What might happen if you wash a throw rug with rubber backing?"
    },
    {
        "query": "What happens if you operate the washer with the detergent drawer removed?"
    },
    {
        "query": "Why should you avoid using dry cleaning detergent in the washer?"
    },
    {
        "query": "What should you do if rust starts forming on the drum?"
    },
    {
        "query": "Why should you avoid using water from cooling or heating devices in the washer?"
    },
    {
        "query": "What is the danger of pressing washer buttons with sharp objects like pins or knives?"
    },
    {
        "query": "Why should pockets be emptied before washing clothes?"
    },
    {
        "query": "Why should socks and brassieres be placed in a washing net?"
    },
    {
        "query": "Why is it important to remove the washer door lever before disposing of the appliance?"
    },
    {
        "query": "What should you do before using the washer for the first time?"
    },
    {
        "query": "Why is it dangerous to wash clothes contaminated with gasoline or kerosene?"
    },
    {
        "query": "Can you open the washer door by force during operation?"
    },
    {
        "query": "Why should you avoid inserting your hand under the washer while it's running?"
    },
    {
        "query": "What is the risk of touching the power plug with wet hands?"
    },
    {
        "query": "What should you do if the water supply hose comes loose and floods the washer?"
    },
    {
        "query": "Why should you unplug the washer during a thunderstorm?"
    },
    {
        "query": "What should you do if a foreign substance enters the washer?"
    },
    {
        "query": "Why is it important to fully insert laundry into the washer?"
    },
    {
        "query": "Why should the water supply hose connectors be checked before using the washer?"
    },
    {
        "query": "Can the washer be used for business purposes?"
    },
    {
        "query": "Why should you not place objects on top of the washer?"
    },
    {
        "query": "Why should volatile materials not be sprayed onto the washer?"
    },
    {
        "query": "Why should you avoid washing waterproof items unless the washer has a special cycle for them?"
    },
    {
        "query": "What happens if the detergent drawer is removed while operating the washer?"
    },
    {
        "query": "Why should you not touch the inside of the drum after a drying cycle?"
    },
    {
        "query": "Why should children not touch the washer door during washing and drying?"
    },
    {
        "query": "Why should you not insert your hand into the detergent drawer?"
    },
    {
        "query": "Can powdered detergent be used in the liquid detergent container?"
    },
    {
        "query": "Why is it unsafe to place animals inside the washer?"
    },
    {
        "query": "What is the danger of pressing washer buttons with sharp objects?"
    },
    {
        "query": "Why should laundry contaminated with skincare oils or lotions not be washed?"
    },
    {
        "query": "Why should metal objects not be left in the drum for long periods?"
    },
    {
        "query": "Why should you not use dry cleaning detergent in the washer?"
    },
    {
        "query": "Why should water from cooling or heating devices not be used in the washer?"
    },
    {
        "query": "Why should socks and brassieres be placed in a washing net?"
    },
    {
        "query": "Why should pockets be emptied before washing clothes?"
    },
    {
        "query": "Why should clothing with large buckles or buttons be sorted carefully?"
    },
    {
        "query": "What should you do before cleaning the washer?"
    },
    {
        "query": "What should I do if my washer generates a burning smell?"
    },
    {
        "query": "Why is it important to remove the washer door lever when disposing of the appliance?"
    },
    {
        "query": "What should I do if my washer gets flooded?"
    },
    {
        "query": "Can I wash items soaked in gasoline or kerosene in the washer?"
    },
    {
        "query": "How can I prevent electric shock when plugging in my washer?"
    },
    {
        "query": "What is the purpose of running Calibration on the washer?"
    },
    {
        "query": "Why should I not install the washer in a location exposed to low temperatures?"
    },
    {
        "query": "What should I do if I suspect a gas leak near my washer?"
    },
    {
        "query": "How should I clean the detergent drawer of my washer?"
    },
    {
        "query": "What is the purpose of Self Clean+ on my washer?"
    },
    {
        "query": "How can I protect my washer during extended periods of non-use?"
    },
    {
        "query": "Why should I not use an electric transformer to power my washer?"
    },
    {
        "query": "How can I safely drain my washer in an emergency?"
    },
    {
        "query": "What is the function of SmartThings in the washer?"
    },
    {
        "query": "Why should I avoid overloading my washer?"
    },
    {
        "query": "What is the purpose of the cycle chart in the manual?"
    },
    {
        "query": "Why is it important to clean the washer\u2019s exterior regularly?"
    },
    {
        "query": "How can I prevent my washer from vibrating excessively?"
    },
    {
        "query": "What does it mean if my washer displays an information code?"
    },
    {
        "query": "Why is it dangerous to use a power strip with my washer?"
    },
    {
        "query": "What are the two conditions required for a device to comply with FCC Part 15 rules?"
    },
    {
        "query": "What should a user do if their equipment is causing harmful interference to radio or TV reception?"
    },
    {
        "query": "What is the FCC radiation exposure requirement for this device?"
    },
    {
        "query": "What channels are available for products sold in the US and Canadian markets?"
    },
    {
        "query": "What does the IC certification number signify?"
    },
    {
        "query": "What precaution should service personnel take when working on certain internal parts of the washer?"
    },
    {
        "query": "What should be done before using the washer for the first time?"
    },
    {
        "query": "What are the recommended actions to reduce the risk of leaks from the washer?"
    },
    {
        "query": "What is the recommended installation height for a washer standpipe?"
    },
    {
        "query": "What is the minimum clearance required for installing a washer in an alcove or closet?"
    },
    {
        "query": "What should be done if a shipping bolt is removed from a washer?"
    },
    {
        "query": "What precautions should be taken when unpacking a washer?"
    },
    {
        "query": "What tools are required for installing the washer?"
    },
    {
        "query": "What type of electrical supply is required for the washer?"
    },
    {
        "query": "What are the FCC compliance requirements for a Class B digital device?"
    },
    {
        "query": "What are the proper grounding requirements for the washer?"
    },
    {
        "query": "What should be done if the washer is to be installed in a location with wood flooring?"
    },
    {
        "query": "What is the maximum Wi-Fi transmitter power for this device?"
    },
    {
        "query": "What is the correct way to obtain the open-source software used in this product?"
    },
    {
        "query": "What should be done if the washer is being moved to a new location?"
    },
    {
        "query": "What is HE laundry detergent used for?"
    },
    {
        "query": "Why is it important to follow detergent manufacturer recommendations?"
    },
    {
        "query": "Can I use powdered detergent in the auto detergent compartment?"
    },
    {
        "query": "What happens if I use non-HE detergent in a HE washer?"
    },
    {
        "query": "What precautions should I take when washing wool items?"
    },
    {
        "query": "What is the purpose of the detergent drawer in a washing machine?"
    },
    {
        "query": "Can I use gel-type or viscous detergent in the detergent drawer?"
    },
    {
        "query": "What is the impact of excessive detergent use?"
    },
    {
        "query": "What should I do if the 'Detergent drawer is open' message appears?"
    },
    {
        "query": "How does the auto-dispensing feature work?"
    },
    {
        "query": "What is the max line in the detergent drawer for?"
    },
    {
        "query": "Why should detergent/softener compartments be emptied before moving the washer?"
    },
    {
        "query": "Can I use organic or natural detergents in the auto dispenser?"
    },
    {
        "query": "What should I do if the 'Refill detergent' message appears?"
    },
    {
        "query": "Can I mix detergent and fabric softener in the same compartment?"
    },
    {
        "query": "How can I increase the scent strength of fabric softener?"
    },
    {
        "query": "What precautions should be taken when using bleach?"
    },
    {
        "query": "Can powdered detergent and powdered oxygen booster be used together?"
    },
    {
        "query": "What should I do if detergent hardens in the compartment?"
    },
    {
        "query": "How do I manually add detergent if auto dispensing is off?"
    },
    {
        "query": "What is the default temperature setting for the 'Steam Whites' cycle?"
    },
    {
        "query": "Which washing cycles support the 'Super Speed' option?"
    },
    {
        "query": "What happens when 'Extra Rinse' is enabled?"
    },
    {
        "query": "Which cycles allow the 'Pre Soak' option?"
    },
    {
        "query": "What are the possible values for 'Dose per Wash' setting?"
    },
    {
        "query": "What is the default setting for 'Dose per Rinse'?"
    },
    {
        "query": "Can I remove the 'Normal' cycle from 'My Cycles' list?"
    },
    {
        "query": "How long does the 'Drum Light' stay on?"
    },
    {
        "query": "How do I set a delayed wash cycle?"
    },
    {
        "query": "What are the different models available in the i.Series\u2122 and Horizon Series\u2122 refrigerators?"
    },
    {
        "query": "What safety precautions should be taken while using the refrigerator?"
    },
    {
        "query": "What is the recommended temperature control range for these refrigerators?"
    },
    {
        "query": "How should temperature probes be handled and maintained?"
    },
    {
        "query": "What are the installation requirements for these refrigerators?"
    },
    {
        "query": "What are the key components located at the rear of the refrigerator?"
    },
    {
        "query": "How can users control the sound for audible alarms?"
    },
    {
        "query": "What should be done before moving the refrigerator?"
    },
    {
        "query": "What are the compliance certifications for these refrigerators?"
    },
    {
        "query": "How should items be stored in the refrigerator?"
    },
    {
        "query": "What should I do if my refrigerator is making a loud noise?"
    },
    {
        "query": "How often should I replace the water filter in my refrigerator?"
    },
    {
        "query": "Can I store hot food directly in the refrigerator?"
    },
    {
        "query": "What is the ideal temperature setting for my refrigerator?"
    },
    {
        "query": "How do I reset the temperature settings on my refrigerator?"
    },
    {
        "query": "Why is there condensation inside my refrigerator?"
    },
    {
        "query": "How can I maximize energy efficiency for my refrigerator?"
    },
    {
        "query": "What should I do if my refrigerator stops cooling?"
    },
    {
        "query": "Is it normal for my refrigerator to run continuously?"
    },
    {
        "query": "How do I safely clean the inside of my refrigerator?"
    },
    {
        "query": "What should be done before powering on the refrigerator?"
    },
    {
        "query": "Why does the refrigerator run frequently when first powered on?"
    },
    {
        "query": "What is the purpose of the backup battery in the refrigerator?"
    },
    {
        "query": "How can you mute an active alarm on the refrigerator?"
    },
    {
        "query": "What must be checked before storing items in the refrigerator?"
    },
    {
        "query": "How can you prevent unauthorized access to items stored in the refrigerator?"
    },
    {
        "query": "What does the Access Control system do?"
    },
    {
        "query": "Can drawers, shelves, and baskets be moved or replaced?"
    },
    {
        "query": "What is the maximum load capacity for single and double-door units?"
    },
    {
        "query": "Where can you find instructions for changing temperature setpoints?"
    },
    {
        "query": "What happens if the Home screen is not interacted with for two minutes?"
    },
    {
        "query": "What does the temperature graph screensaver display?"
    },
    {
        "query": "How are active alarms displayed on the screen?"
    },
    {
        "query": "What happens when multiple alarms are active?"
    },
    {
        "query": "How can audible alarms be controlled?"
    },
    {
        "query": "Where is the light control for the chamber located?"
    },
    {
        "query": "How often should the condenser grill be cleaned?"
    },
    {
        "query": "What are the available input voltage options for the refrigerator?"
    },
    {
        "query": "What is the maximum load capacity for remote alarm interface terminals?"
    },
    {
        "query": "What is the weight of an iB 245 model refrigerator?"
    },
    {
        "query": "What should be done before turning on the refrigerator for the first time?"
    },
    {
        "query": "What is the function of the backup battery in the refrigerator?"
    },
    {
        "query": "How can you temporarily mute an alarm on the refrigerator?"
    },
    {
        "query": "Why is reduceByKey more efficient than groupByKey in Spark?"
    },
    {
        "query": "How does the Access Control option enhance refrigerator security?"
    },
    {
        "query": "Where can you find information about active alarms on the refrigerator?"
    },
    {
        "query": "How does the temperature graph screensaver function?"
    },
    {
        "query": "What should be done before storing items in the refrigerator?"
    },
    {
        "query": "What is the recommended maintenance schedule for testing alarms?"
    },
    {
        "query": "What is the maximum load capacity for a single drawer?"
    },
    {
        "query": "What happens if the backup battery is not functioning during a power failure?"
    },
    {
        "query": "What are the available input voltage options for the refrigerator?"
    },
    {
        "query": "What should you do if an alarm sounds when powering on the refrigerator?"
    },
    {
        "query": "What is the purpose of the condensate evaporator tray?"
    },
    {
        "query": "What should you consider when cleaning the refrigerator?"
    },
    {
        "query": "How does the refrigerator handle unauthorized access attempts?"
    },
    {
        "query": "Where is the power switch located on the refrigerator?"
    },
    {
        "query": "What should be done if the refrigerator needs to be moved?"
    },
    {
        "query": "How does the refrigerator display temperature data?"
    },
    {
        "query": "What should be done if the backup battery has been in use for two years?"
    },
    {
        "query": "What should be done before powering on the refrigerator for the first time?"
    },
    {
        "query": "How does the refrigerator handle power failure?"
    },
    {
        "query": "What happens when the refrigerator reaches normal operating temperature after first-time power on?"
    },
    {
        "query": "How can an alarm be muted when it sounds during power-on?"
    },
    {
        "query": "What should be checked before storing items in the refrigerator?"
    },
    {
        "query": "How can unauthorized access to stored items be prevented?"
    },
    {
        "query": "What is the Access Control option and how does it work?"
    },
    {
        "query": "Can drawers, shelves, and baskets be removed or replaced?"
    },
    {
        "query": "Can the temperature controller setpoints be changed?"
    },
    {
        "query": "How can the temperature monitor offset value be adjusted?"
    },
    {
        "query": "What is the factory preset hysteresis value for blood bank models?"
    },
    {
        "query": "What are the default high and low temperature alarm setpoints?"
    },
    {
        "query": "How can temperature alarm setpoints be changed?"
    },
    {
        "query": "What visual alarms does the refrigerator have?"
    },
    {
        "query": "How can audible alarms be muted?"
    },
    {
        "query": "Where is the light switch located?"
    },
    {
        "query": "What maintenance tasks should be performed quarterly?"
    },
    {
        "query": "How often should the backup battery for the chart recorder be checked?"
    },
    {
        "query": "What is the power consumption for model 125 at 115V, 60Hz?"
    },
    {
        "query": "What is the maximum drawer load capacity?"
    },
    {
        "query": "What should be done before powering on the refrigerator for the first time?"
    },
    {
        "query": "Why does the refrigerator run frequently when powered on for the first time?"
    },
    {
        "query": "What should be done if an alarm sounds upon initial power-up?"
    },
    {
        "query": "How is the backup battery prepared for use?"
    },
    {
        "query": "What precaution should be taken with the condensate evaporator tray?"
    },
    {
        "query": "How do you store items safely in the refrigerator?"
    },
    {
        "query": "Why should the refrigerator chamber temperature stabilize before storing products?"
    },
    {
        "query": "How can unauthorized access to stored items be prevented?"
    },
    {
        "query": "What is the purpose of the Access Control system?"
    },
    {
        "query": "What should be checked before moving drawers or shelves?"
    },
    {
        "query": "Can the factory preset temperature setpoints be changed?"
    },
    {
        "query": "What is the allowed range for the Monitor Offset setting?"
    },
    {
        "query": "Should the Control Offset be modified?"
    },
    {
        "query": "What is the function of hysteresis in temperature control?"
    },
    {
        "query": "What is the default high temperature alarm setpoint?"
    },
    {
        "query": "What is the default low temperature alarm setpoint for pharmacy and laboratory models?"
    },
    {
        "query": "How do you enter program mode to change temperature setpoints?"
    },
    {
        "query": "How do you identify an active visual alarm?"
    },
    {
        "query": "What should be done if the door is left open for more than three minutes?"
    },
    {
        "query": "How can audible alarms be muted?"
    },
    {
        "query": "Where is the chamber light switch located?"
    },
    {
        "query": "How often should the condenser grill be cleaned?"
    },
    {
        "query": "How frequently should the high and low temperature alarms be tested?"
    },
    {
        "query": "What maintenance is required after an extended power failure?"
    },
    {
        "query": "What should be done if the backup battery does not function during an alarm test?"
    },
    {
        "query": "What is the voltage tolerance for the refrigerator models?"
    },
    {
        "query": "What are the available input voltage and frequency options for the refrigerator?"
    },
    {
        "query": "What is the power consumption for model 125 at 230V 50Hz?"
    },
    {
        "query": "What is the weight of model HLR 120?"
    },
    {
        "query": "What is the maximum drawer load capacity?"
    }
]

In [0]:
# import pickle



# # Save the list to a file
# with open("eval_set.pkl", "wb") as f:
#     pickle.dump(eval_set, f)
# with open("eval_set.pkl", "rb") as f:
#     loaded_list = pickle.load(f)

# print(len(loaded_list)) 
len(eval_set)

259


259

In [0]:
import mlflow
import pandas as pd

# Define test examples
eval_set_df = pd.DataFrame({
    "request": eval_set
    })

# Define global guidelines
global_guidelines = {
  "english": ["The response must be in English"],
  "clarity": ["The response must be clear, coherent, and concise"],
}

# Ensure `loaded_model` is defined before calling mlflow.evaluate
evaluation_results = mlflow.evaluate(
    model=loaded_model,  # Ensure this variable is defined
    data=eval_set_df,
    model_type="databricks-agent",  # Replace with your model type
    evaluator_config={
       "databricks-agent": {"global_guidelines": global_guidelines}
    }
)

display(evaluation_results)


Evaluating:   0%|          | 0/259 [Elapsed: 00:00, Remaining: ?]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-63dc9d33-c60f-4d04-b330-11c04c89ff54/lib/python3.12/site-packages/databricks/sdk/errors/base.py:87: UserWarning: The 'retry_after_secs' parameter of DatabricksError is deprecated and will be removed in a future version.
  warnings.warn(


<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 100/259 pass